In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from darts import TimeSeries
from darts.utils.missing_values import extract_subseries
from sklearn.metrics import mean_squared_error
from ipywidgets import interact, SelectMultiple, IntSlider



In [2]:
data = pd.read_csv("../processed/data.csv", index_col=0, parse_dates=True)
data.rename(columns={"FB20F11_81": "flow"}, inplace=True)
data[["flow", "acc_precip"]] = data[["flow", "acc_precip"]].apply(np.log1p)
data.drop(columns=["temp_grass", "temp_soil_30"], inplace=True)

ts = TimeSeries.from_dataframe(data, freq="h")

subseries = extract_subseries(ts, mode="any")
min_length = 24 * 7  # 7 days
subseries = [s for s in subseries if len(s) >= min_length]



In [3]:
train_index = list(range(0, 2)) + list(range(3, len(subseries)))
test_index = [2]

train_subseries = [subseries[i] for i in train_index]
test_subseries = [subseries[i] for i in test_index]

train_df = pd.concat([s.pd_dataframe() for s in train_subseries])
test_df = test_subseries[0].pd_dataframe()

train_df.index.name = "time"
test_df.index.name = "time"


In [4]:
def add_fourier_terms(df, period=24):
    timestamps = (df.index - df.index[0]).total_seconds() / 3600  # Convert to hours
    omega = 2 * np.pi / period
    df['fourier_sin'] = np.sin(omega * timestamps)
    df['fourier_cos'] = np.cos(omega * timestamps)
    return df

train_df = add_fourier_terms(train_df, period=24)
test_df = add_fourier_terms(test_df, period=24)


In [ ]:
def create_future_dataset(df, max_horizon=24):
    expanded_data = []
    for horizon in range(1, max_horizon + 1):
        df_copy = df.copy()
        df_copy["horizon"] = horizon 
        df_copy["flow_target"] = df["flow"].shift(-horizon)

        expanded_data.append(df_copy)

    full_future_df = pd.concat(expanded_data)
    
    return full_future_df.dropna()

def create_future_dataset_with_future_precip(df, max_horizon=24, max_future_precip=24):
    # First determine how many rows will be lost due to the highest horizon and future precip
    max_shift = max(max_horizon, max_future_precip)
    
    # Truncate the dataframe first to avoid different NaN patterns per horizon
    truncated_df = df.iloc[:-max_shift].copy() if len(df) > max_shift else df.copy()
    
    expanded_data = []
    
    for horizon in range(1, max_horizon + 1):
        df_copy = truncated_df.copy()
        df_copy["horizon"] = horizon 
        df_copy["flow_target"] = truncated_df["flow"].shift(-horizon)
        
        # Add future precipitation data with various lead times
        for future_step in range(1, max_future_precip + 1):
            if future_step <= horizon:  # Only add future precip that would be available at prediction time
                df_copy[f"future_precip_{future_step}"] = truncated_df["acc_precip"].shift(-future_step)
        
        # Drop NaNs for this horizon specifically before adding to the list
        df_copy = df_copy.dropna()
        expanded_data.append(df_copy)

    full_future_df = pd.concat(expanded_data)
    
    return full_future_df

In [6]:
 
train_future_df = create_future_dataset_with_future_precip(train_df) # use this one for future precipitation
test_future_df = create_future_dataset_with_future_precip(test_df)

# train_future_df = create_future_dataset(train_df)
# test_future_df = create_future_dataset(test_df)

X_train_future = train_future_df.drop(columns=["flow", "flow_target"])
y_train_future = train_future_df["flow_target"]
X_test_future = test_future_df.drop(columns=["flow", "flow_target"])
y_test_future = test_future_df["flow_target"]



In [7]:
scaling_features = [col for col in X_train_future.columns if col != "horizon"]
scaler = StandardScaler()
X_train_scaled = X_train_future.copy()
X_train_scaled[scaling_features] = scaler.fit_transform(X_train_future[scaling_features])
X_test_scaled = X_test_future.copy()
X_test_scaled[scaling_features] = scaler.transform(X_test_future[scaling_features])

X_train_scaled["horizon"] = X_train_future["horizon"]
X_test_scaled["horizon"] = X_test_future["horizon"]
print(X_train_scaled["horizon"].value_counts())

horizon
24    6937
Name: count, dtype: int64


In [8]:
multi_step_model = xgb.XGBRegressor(objective="reg:squarederror")
multi_step_model.fit(X_train_scaled, y_train_future)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [9]:
def forecast_all_horizons(model, X_test_scaled, max_horizon=24):
    predictions = {}
    
    for horizon in range(1, max_horizon + 1): 
        X_test_horizon = X_test_scaled[X_test_scaled["horizon"] == horizon]
        
        if X_test_horizon.empty:
            print(f"Skipping horizon {horizon} due to no available data.")
            continue

        y_pred = model.predict(X_test_horizon)
        
        # Store predictions with the same index as the original data
        predictions[horizon] = pd.Series(y_pred, index=X_test_horizon.index)

    # Convert dictionary to DataFrame with horizons as columns
    return pd.DataFrame(predictions)

In [10]:
y_pred_all_horizons = forecast_all_horizons(multi_step_model, X_test_scaled)

print(f"Predictions Shape: {y_pred_all_horizons.shape}, Test Shape: {y_test_future.shape}")


Skipping horizon 1 due to no available data.
Skipping horizon 2 due to no available data.
Skipping horizon 3 due to no available data.
Skipping horizon 4 due to no available data.
Skipping horizon 5 due to no available data.
Skipping horizon 6 due to no available data.
Skipping horizon 7 due to no available data.
Skipping horizon 8 due to no available data.
Skipping horizon 9 due to no available data.
Skipping horizon 10 due to no available data.
Skipping horizon 11 due to no available data.
Skipping horizon 12 due to no available data.
Skipping horizon 13 due to no available data.
Skipping horizon 14 due to no available data.
Skipping horizon 15 due to no available data.
Skipping horizon 16 due to no available data.
Skipping horizon 17 due to no available data.
Skipping horizon 18 due to no available data.
Skipping horizon 19 due to no available data.
Skipping horizon 20 due to no available data.
Skipping horizon 21 due to no available data.
Skipping horizon 22 due to no available dat

In [11]:
print(y_pred_all_horizons.shape, y_test_future.shape)

(1110, 1) (1110,)


In [12]:
forecast_horizon = 24  
rmse_scores = []

for step in range(1, forecast_horizon + 1):
    # Get actual values for this horizon
    horizon_mask = X_test_future["horizon"] == step
    y_true = y_test_future[horizon_mask]
    
    # Get predicted values for this horizon
    if step in y_pred_all_horizons.columns:
        # Get predictions and align with true values by index
        y_pred = y_pred_all_horizons[step].reindex(y_true.index)
        
        valid_idx = y_true.notna() & y_pred.notna()
        y_true = y_true[valid_idx]
        y_pred = y_pred[valid_idx]

        if len(y_true) > 0:
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        else:
            rmse = np.nan
    else:
        print(f"No predictions for horizon {step}")
        rmse = np.nan

    rmse_scores.append(rmse)

No predictions for horizon 1
No predictions for horizon 2
No predictions for horizon 3
No predictions for horizon 4
No predictions for horizon 5
No predictions for horizon 6
No predictions for horizon 7
No predictions for horizon 8
No predictions for horizon 9
No predictions for horizon 10
No predictions for horizon 11
No predictions for horizon 12
No predictions for horizon 13
No predictions for horizon 14
No predictions for horizon 15
No predictions for horizon 16
No predictions for horizon 17
No predictions for horizon 18
No predictions for horizon 19
No predictions for horizon 20
No predictions for horizon 21
No predictions for horizon 22
No predictions for horizon 23


In [13]:
import numpy as np
import matplotlib.pyplot as plt

def mape(y_true_log, y_pred_log):
    y_true_original = np.expm1(y_true_log)
    y_pred_original = np.expm1(y_pred_log)
    return np.mean(np.abs((y_true_original - y_pred_original) / y_true_original)) * 100
forecast_horizon = 24  
mape_scores = []

for step in range(1, forecast_horizon + 1):
    y_true = y_test_future[X_test_future["horizon"] == step]
    y_pred = y_pred_all_horizons[step].reindex(y_true.index) 

    valid_idx = y_true.notna() & y_pred.notna()
    y_true = y_true[valid_idx]
    y_pred = y_pred[valid_idx]

    if len(y_true) > 0 and (y_true != 0).all():
        mape_value = mape(y_true, y_pred)
    else:
        mape_value = np.nan  

    mape_scores.append(mape_value)

steps = np.arange(1, forecast_horizon + 1)

plt.figure(figsize=(10, 5))
plt.scatter(steps, mape_scores, c=mape_scores, cmap='coolwarm', edgecolors='black', s=100)
plt.plot(steps, mape_scores, linestyle='--', color='gray', alpha=0.7)

plt.xlabel("Forecast Horizon (t+ step)")
plt.ylabel("Mean Absolute Percentage Error (MAPE)")
plt.title("MAPE Across Forecast Horizon")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


KeyError: 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, IntSlider

feature_i = SelectMultiple(
    options=X_test_scaled.columns,
    value=[X_test_scaled.columns[0]], 
    description="Features to compare",
)

nudger = IntSlider(min=-12, max=12, value=0, description="Nudge pred. start")

@interact(features=feature_i, nudge=nudger)
def plot_forecast(features, nudge):
    arbitrary_breakpoints = [50, 100, 300, 500]  
    fig, axes = plt.subplots(2, len(arbitrary_breakpoints), figsize=(20, 10), sharey="row")
    
    unique_timestamps = test_df.index.unique()
    
    for i, n_split in enumerate(arbitrary_breakpoints):
        idx = min(n_split + nudge, len(unique_timestamps) - 25)  
        
        if idx < 0:
            continue
            
        base_time = unique_timestamps[idx]
        
        forecast_times = unique_timestamps[idx:idx+24]
        
        actual_series = []
        for t in forecast_times:
            actual_idx = test_df.index.get_loc(t)
            actual_series.append(test_df.iloc[actual_idx]['flow'])
            
        pred_series = []
        for h, t in enumerate(forecast_times, 1):
            mask = (X_test_future.index == base_time) & (X_test_future['horizon'] == h)
            if any(mask):
                pred_value = y_pred_all_horizons.loc[base_time, h] if h in y_pred_all_horizons.columns else np.nan
                pred_series.append(pred_value)
            else:
                pred_series.append(np.nan)
        
        axes[0, i].plot(forecast_times, actual_series, label="Actual", color="blue", marker="o", alpha=0.7)
        axes[0, i].plot(forecast_times, pred_series, label="Predicted", color="red", marker="x", alpha=0.7)
        axes[0, i].set_title(f"Forecast starting at {base_time} (Segment {i+1})")
        axes[0, i].axvline(base_time, color="gray", linestyle="--", alpha=0.6, label="Forecast Start")
        axes[0, i].legend()
        axes[0, i].grid(True)
        axes[0, i].set_xticks(forecast_times[::4]) 
        axes[0, i].tick_params(axis='x', rotation=45)
        
        for feature in features:
            if feature in X_test_scaled.columns:
                feature_series = []
                for t in forecast_times:
                    mask = X_test_scaled.index == t
                    if any(mask):
                        feature_series.append(X_test_scaled.loc[mask, feature].mean())
                    else:
                        feature_series.append(np.nan)
                
                axes[1, i].plot(forecast_times, feature_series, label=feature)
                
        axes[1, i].set_xlabel("Time")
        axes[1, i].set_ylabel("Feature Value")
        axes[1, i].legend()
        axes[1, i].grid(True)
        axes[1, i].set_xticks(forecast_times[::4]) 
        axes[1, i].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

interactive(children=(SelectMultiple(description='Features to compare', index=(0,), options=('acc_precip', 'me…